<a href="https://colab.research.google.com/github/eliashahbazi/VideoStimulus/blob/master/Percepto_Copy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
module_path = 'https://tfhub.dev/deepmind/biggan-deep-256/1'  # 256x256 BigGAN-deep
import IPython.display
import numpy as np
import PIL.Image
from scipy.stats import truncnorm
import tensorflow as tf
import tensorflow_hub as hub
import io

from pytorch_pretrained_biggan import BigGAN
from torchvision.transforms import ToPILImage, Resize, Compose
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

from numpy import genfromtxt
from IPython.display import display
import ipywidgets as widgets
import random
from imagenet_clsidx_to_labels import idx_to_name_dict
import torch

ModuleNotFoundError: ignored

In [1]:
module_path = 'https://tfhub.dev/deepmind/biggan-deep-256/1'  # 256x256 BigGAN-deep
import io
import IPython.display
import numpy as np
import PIL.Image
from scipy.stats import truncnorm
import tensorflow as tf
from pytorch_pretrained_biggan import BigGAN
from torchvision.transforms import ToPILImage, Resize, Compose
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

from numpy import genfromtxt
from IPython.display import display
import ipywidgets as widgets
import random
from imagenet_clsidx_to_labels import idx_to_name_dict
import torch

def BG_dist(idx1,idx2,alpha,truncation):
       def get_c_vector(idx1: int, idx2: int, alpha: float, batch_size: int=1):
           c = torch.zeros(batch_size, 1000)
           c[:, idx1] = 1-alpha
           c[:, idx2] = alpha
           return c

       def get_z_vector(truncation: float, batch_size: int=1):
           values = truncnorm.rvs(-2, 2, size=(batch_size, 128)).astype(np.float32)
           values = torch.from_numpy(values)
           return values * truncation

       
        
       c = get_c_vector(idx1, idx2, alpha, batch_size=1)
       z = get_z_vector(truncation, batch_size=1)

       #print('C size =',c.size(),' Z size=',z.size())
       #print(z) 
       print('Generating an image ...')
       model = BigGAN.from_pretrained(f'biggan-deep-{512}')
       with torch.no_grad():
           out = model(z, c, truncation)

       print('Image generated.')
       final_transforms = Compose([
           ToPILImage(),
           Resize((1024, 1024))
       ])

       out = out.squeeze()
       out = out/2 + 0.5
       final = final_transforms(out)
       return final,c,z
       
 
def manual_dist(c,z,truncation):
       model = BigGAN.from_pretrained(f'biggan-deep-{512}')
       with torch.no_grad():
           out = model(z, c, truncation)

       #print('Image generated.')
       final_transforms = Compose([
           ToPILImage(),
           Resize((1024, 1024))
       ])

       out = out.squeeze()
       out = out/2 + 0.5
       final = final_transforms(out)
       return final 



def img_show(imlist):
    rand = random.randrange(0,len(imlist))
    print(int(imlist[rand,5]) ,int(imlist[rand,6]),imlist[rand,7],imlist[rand,8])
    [final,c,z] = BG_dist(int(imlist[rand,5]) ,int(imlist[rand,6]),imlist[rand,7],imlist[rand,8])
    truncation = imlist[rand,8]
    #plt.imshow(final)
    #print(z[0,1])
    return c,z,truncation
    
def on_button_clicked(b):
    with output:
        z[0,0]  = torch.FloatTensor([p.value])
        z[0,10] = torch.FloatTensor([q.value])
        z[0,100]= torch.FloatTensor([r.value])
        nonzeroind = np.nonzero(torch.Tensor.numpy(c))[1]
         
        print(torch.Tensor.numpy(c[:,nonzeroind[0]]))
        alpha = torch.Tensor.numpy(c[:,nonzeroind[0]])
        idx1 = nonzeroind[0]
        idx2 = nonzeroind[1]
        
        c[:,idx1]= torch.FloatTensor([alpha /2])
        c[:,idx2] = torch.FloatTensor([alpha - (alpha/2)])
        c[:,int(box1.value)]= torch.FloatTensor([(1-alpha)/2])
        c[:,int(box2.value)]= torch.FloatTensor([(1-alpha)-(1-alpha)/2])
        
        
        img = manual_dist(c,z,t)
        f = BytesIO()
        img.save(f,'png')
        IPython.display.display(IPython.display.Image(data=f.getvalue(), width=200))
        return f

def view_image(i):
    
    #[c,z,truncation]=img_show(imlist)
    z[0,0]=i
    
def box1(x):
    return x
    
dtype = [('id', np.int32),('b_idx1',np.int32),('b_idx2',np.int32),('b_alpha',np.float64),('b_t',np.float64),('f_idx1',np.int32),('f_idx2',np.int32),('f_alpha',np.float64),('f_t',np.float64),('trans',np.float64),('dir',np.int32)]
imlist = np.zeros(1, dtype=dtype)

imlist = genfromtxt('human/psytest.csv',delimiter=",")

[c,z,t]=img_show(imlist)
#print(c)

button = widgets.Button(description="Click Me!")
output = widgets.Output()

p = widgets.FloatSlider()
p.min = z[0,0]-5
p.max = z[0,0]+5
p.value = z[0,0]
p.step = 0.1
display(p)

q = widgets.FloatSlider()
q.min = z[0,10]-5
q.max = z[0,10]+5
q.value = z[0,10]
q.step = 0.1
display(q)

r = widgets.FloatSlider()
r.min = z[0,100]-5
r.max = z[0,100]+5
r.value = z[0,100]
r.step = 0.1
display(r)

box1=widgets.Text(value='10')
box2=widgets.Text(value='700')
display(box1)
display(box2)

display(button, output)
button.on_click(on_button_clicked)




ModuleNotFoundError: ignored

# New Section